# Vector Space Retrieval

Vector Space Retrieval in text retrieval converts documents and queries into vectors, where each term represents a dimension. The frequency of terms in a document (e.g., TF-IDF) determines the value for each vector dimension. To find relevant documents, the system calculates the similarity between the query vector and document vectors, usually using cosine similarity. The documents with the highest similarity scores are considered the most relevant. This method enables ranked retrieval by measuring how close vectors are, rather than relying solely on exact matches.

In [18]:
# We need a list where we can store all the extract text from pdf
documents = []

In [19]:
# Extract text from PDF files
import pdfplumber


def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text


docs_collection = ['computer_science.pdf', 'physics.pdf', 'art.pdf',
                   'football_history.pdf', 'artificial_intelligence.pdf']


for path in docs_collection:
  # For every document in the collection we fetch the text and store
  doc = extract_text_from_pdf(path)
  documents.append(doc)


In [20]:
import spacy
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Step 2: Preprocess and create a vocabulary
vocab = set()
preprocessed_docs = []

for doc in documents:
    # Apply spaCy NLP pipeline to the document
    doc_nlp = nlp(doc)
    # Filter and lemmatize tokens, excluding stop words and certain POS tags
    filtered_tokens = [
        token.lemma_.upper() for token in doc_nlp
        if not token.is_stop and token.pos_ not in {"DET", "ADP", "AUX", "CCONJ", "ADJ", "PUNCT", "SPACE"}
    ]
    # Update vocabulary set with filtered tokens
    # Vocab keeps only unique tokens
    vocab.update(filtered_tokens)
    preprocessed_docs.append(filtered_tokens)  # Append filtered tokens to preprocessed documents list

# Sorted list for consistent vector representation
vocab = sorted(vocab)

# Step 3: Compute term frequency (TF)
def compute_tf(tokens, vocab):
    term_counts = Counter(tokens)
    # Number of times term t appears in document d / total number of terms in the doc d
    return np.array([term_counts[term] / len(tokens) for term in vocab])

doc_vectors = [compute_tf(tokens, vocab) for tokens in preprocessed_docs]

# Step 4: Compute inverse document frequency (IDF)
def compute_idf(docs, vocab):
    # log(total number of docs / number of docs containing term t)
    idf = np.log(len(docs) / (1 + np.array([sum(1 for doc in docs if term in doc) for term in vocab])))
    return idf

idf = compute_idf(preprocessed_docs, vocab)
tfidf_doc_vectors = [tf_vec * idf for tf_vec in doc_vectors]

# Step 5: Create TF-IDF query vector
def get_query_vector(query, vocab):
    # Lemmatized tokens of the query
    query_tokens = [token.lemma_.upper() for token in nlp(query)]
    query_tf = compute_tf(query_tokens, vocab)
    return query_tf * idf

# Step 6: Calculate cosine similarity
def calculate_similarity(query_vector, doc_vector):
    return cosine_similarity([query_vector], [doc_vector])[0][0]

# Step 7: Search documents using vector space model
def search_docs(query, tfidf_doc_vectors, vocab):
    query_vector = get_query_vector(query, vocab)
    similarities = []

    for doc_index, doc_vector in enumerate(tfidf_doc_vectors):
        sim_score = calculate_similarity(query_vector, doc_vector)
        if sim_score > 0:
            similarities.append({'doc_index': doc_index, 'doc_score': sim_score})

    similarities.sort(key=lambda x: x['doc_score'], reverse=True)

    for doc in similarities:
        print(doc)


In [22]:
search_docs("quantum", tfidf_doc_vectors, vocab)

{'doc_index': 1, 'doc_score': 0.5179971042532392}
{'doc_index': 0, 'doc_score': 0.044226915215261814}


In [23]:
search_docs("history", tfidf_doc_vectors, vocab)

{'doc_index': 3, 'doc_score': 0.05068685199546379}
{'doc_index': 2, 'doc_score': 0.03368341028251341}


In [26]:
search_docs("ai", tfidf_doc_vectors, vocab)

{'doc_index': 4, 'doc_score': 0.4716339957973389}
{'doc_index': 0, 'doc_score': 0.13268074564578544}


In [28]:
search_docs("science artificial intelligence", tfidf_doc_vectors, vocab)

{'doc_index': 0, 'doc_score': 0.30641305685949793}
{'doc_index': 4, 'doc_score': 0.14852618969049403}
{'doc_index': 2, 'doc_score': 0.019447125993833726}


In [29]:
search_docs("artificial intelligence", tfidf_doc_vectors, vocab)

{'doc_index': 4, 'doc_score': 0.18190668909076688}
{'doc_index': 0, 'doc_score': 0.09381945497902239}


# Conclusion

All the queries returned accurate results. If we check the PDF documents, we can confirm that the search effectively identifies and ranks the most relevant documents.

### Advantages

- Extreme simple an intuitive query model.
- Allows partial matches (documents don't need all query terms)

### Disadvantages

- Can be biased by term spamming.
- Assumes term independence, which isn't always true (e.g., synonyms).

# BM25 (Best Matching 25)

We have seen the effectiveness of Vector Space Retrieval, but it also comes with issues like bias from term spamming, which is something we want to avoid. For example, if a document has the term "AI" repeated 100 times, it might be ranked first due to the high term frequency, even if it's not truly the most relevant.

To address this issue, we can use BM25, which is a ranking function designed to tackle two main problems with traditional TF-IDF:

- **Diminishing returns on term frequency**: BM25 uses a saturation function so that repeating a term excessively doesn’t lead to an exaggerated boost in relevance.
- **Document length normalization**: BM25 adjusts for document length, making sure shorter documents are not unfairly ranked lower than longer ones.
This helps ensure that relevance scores are more balanced and reflective of actual content quality.